<a href="https://colab.research.google.com/github/kishor-cypto/projects/blob/main/ArsenicProject_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [2]:
dataset_path = "/content/drive/MyDrive/dataset/Augmented"

img_size = (128, 128)
batch_size = 32

train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

test_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
      image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)

Found 8893 files belonging to 2 classes.
Using 7115 files for training.
Found 8893 files belonging to 2 classes.
Using 1778 files for validation.
Classes: ['infected', 'not_infected']


In [3]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [4]:
model = Sequential([
    Flatten(input_shape=(128,128,3)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 49152)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    25,166,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,330,818 (96.63 MB)

 Trainable params: 25,330,818 (96.63 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)


Epoch 1/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 1178s 5s/step - accuracy: 0.5541 - loss: 3.7070 - val_accuracy: 0.5866 - val_loss: 0.8180
Epoch 2/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 149s 664ms/step - accuracy: 0.6237 - loss: 0.6973 - val_accuracy: 0.6755 - val_loss: 0.6040
Epoch 3/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 149s 668ms/step - accuracy: 0.6388 - loss: 0.6445 - val_accuracy: 0.6327 - val_loss: 0.6717
Epoch 4/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 173s 775ms/step - accuracy: 0.6551 - loss: 0.6078 - val_accuracy: 0.6552 - val_loss: 0.5855
Epoch 5/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 187s 709ms/step - accuracy: 0.6557 - loss: 0.5964 - val_accuracy: 0.6760 - val_loss: 0.5830
Epoch 6/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 155s 695ms/step - accuracy: 0.6390 - loss: 0.6098 - val_accuracy: 0.6867 - val_loss: 0.6393
Epoch 7/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 156s 698ms/step - accuracy: 0.6399 - loss: 0.6439 - val_accuracy: 0.6502 - val_loss: 0.6365
Epoch 8/10
223/223 ━━━━━━━━━━━━━━━━━━━━ 153s 677ms/step - accuracy: 0.6750 - l

In [6]:
loss, acc = model.evaluate(test_ds)
print("Test Accuracy =", acc)


56/56 ━━━━━━━━━━━━━━━━━━━━ 14s 252ms/step - accuracy: 0.6809 - loss: 0.5848
Test Accuracy = 0.6743531823158264


In [8]:
img_path = "/content/drive/MyDrive/dataset/IMG_1895_augmented_6.png"

img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

prediction = model.predict(img_array)
predicted_class = class_names[np.argmax(prediction)]

print("Prediction:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: infected
